In [16]:
import quimb.tensor as qtn
from vmc_torch.experiment.tn_model import (
    fTNModel,
    fTN_backflow_attn_Tensorwise_Model_v5,
    fTN_backflow_attn_Tensorwise_Model_v1,
)
from vmc_torch.fermion_utils import generate_random_fpeps
import symmray as sr
import torch
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch

Lx=4
Ly=4
N_f = int(Lx*Ly-2)
t=1
U=8
n_fermions_per_spin = (N_f//2, N_f//2)
H = spinful_Fermi_Hubbard_square_lattice_torch(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
# SU in quimb
D = 4
seed = 0
symmetry = 'Z2'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=D, seed=seed, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
try:
    parse_edges_to_site_info = sr.utils.parse_edges_to_site_info
except AttributeError:
    parse_edges_to_site_info = sr.parse_edges_to_site_info
site_info = parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.float32))
peps_model = fTNModel(peps, max_bond=-1)
# model = fTN_backflow_attn_Tensorwise_Model_v5(
#     peps,
#     max_bond=-1,
#     embedding_dim=64,
#     attention_heads=4,
#     nn_final_dim=32,
#     nn_eta=1.0,
# )
model = fTN_backflow_attn_Tensorwise_Model_v1(
    peps,
    max_bond=-1,
    embedding_dim=16,
    attention_heads=4,
    nn_final_dim=4,
    nn_eta=1.0,
)
x_i = torch.tensor(H.hilbert.random_state())
model(x_i)
peps_model.num_params

nn_correction mean&std: 0.011048163287341595, 0.39275968074798584
params_vec mean&std: -0.028523989021778107, 1.001470923423767


  0%|          | 0/10 [00:00<?, ?it/s]

F=4.76 C=5.57 S=10.00 P=11.43: 100%|██████████| 10/10 [00:00<00:00, 425.90it/s]


3200

In [98]:
from vmc_torch.experiment.tn_model import *
# Reconstruct the original parameter structure (by unpacking from the flattened dict)
params = {
    int(tid): {
        ast.literal_eval(sector): data
        for sector, data in blk_array.items()
    }
    for tid, blk_array in peps_model.torch_tn_params.items()
}
params_vec = flatten_tn_params(params)

# `x` is expected to be batched as (batch_size, input_dim)
# Loop through the batch and compute amplitude for each sample
batch_amps = []
x_i = torch.tensor([0,1,1,0, 1,0,1,0, 0,1,0,1, 1,0,1,0], dtype=torch.int8)[:Lx*Ly]
# Check x_i type
if not isinstance(x_i, torch.Tensor):
    x_i = torch.tensor(x_i, dtype=peps_model.param_dtype)
else:
    if x_i.dtype != peps_model.param_dtype:
        x_i = x_i.to(peps_model.param_dtype)
    tn_nn_params = reconstruct_tn_params(params_vec, params)
psi = qtn.unpack(tn_nn_params, peps_model.skeleton)
# Get the amplitude
amp = psi.get_amp(x_i)
print(amp.contract())
for tid,ts in amp.tensor_map.items():
    ts_param = ts.get_params()
    ts_param_vec = flatten_fts_params(ts_param)
    new_ts_param_vec = ts_param_vec + 0.01*torch.randn_like(ts_param_vec)
    new_ts_param = reconstruct_fts_params(new_ts_param_vec, ts_param)
    ts.set_params(new_ts_param)
print(amp.contract())

tensor(-145362.3750, grad_fn=<MulBackward0>)
tensor(-140103.3125, grad_fn=<MulBackward0>)


In [2]:
D8_peps = generate_random_fpeps(Lx, Ly, D=4, seed=seed, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
D8_peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.float32))
peps_nn_model = fTN_backflow_attn_Tensorwise_Model_v1(D8_peps, max_bond=-1, embedding_dim=16, attention_heads=4, nn_final_dim=8)
peps_nn_model.num_params

15824

In [10]:
dtype = torch.float64
model = NNBF_attention(
    nsite=Lx*Ly,
    hilbert=H.hilbert,
    param_dtype=dtype,
    hidden_dim=64,
    embed_dim=16,
    nn_eta=0.0,
    attention_heads=4,
    phys_dim=4,
    spinflip_symmetry=False,
)

model.num_params

15888

In [9]:
sd = SlaterDeterminant(
    H.hilbert,
    param_dtype=dtype,
)
sd.num_params

96